In [1]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from tqdm import tqdm, tqdm_notebook
from xgboost import XGBRegressor
import warnings, sys, os, gc

## Carga Parcial

In [6]:
#df_train = pd.read_csv('./data/opiniones_train.csv')
#df_test = pd.read_csv('./data/opiniones_test.csv')
#df_libros = pd.read_csv('./data/libros.csv')
df_train = pd.read_csv('./data/train_svd.csv')
df_test = pd.read_csv('./data/test_svd.csv')
print(df_train.shape, df_test.shape)

(42320, 5) (10584, 6)


## Preprocesamiento

In [20]:
def write_submit(df_test,prediccion,fileName):
    submit = pd.DataFrame({'id':df_test.id})
    submit['puntuacion'] = pd.DataFrame(np.around(prediccion, 2))
    submit.head()
    len(submit)
    submit.to_csv("./submission/"+fileName+".csv", index=False)

In [3]:
## Cambio sring del nombre del libro por isbn
## Elimino libros con valores na,"Español", "/", "B" y "q"
#train = df_train
#print(len(train))
#train['libro'] = train['libro'].map(df_libros.set_index('libro')['isbn'])
#train = train[train['libro'].notna()]
#train = train[~train.libro.str.contains("Español")]
#train = train[~train.libro.str.contains("/")]
#train = train[~train.libro.str.contains("B")]
#train = train[~train.libro.str.contains("-")]
#train = train[~train.libro.str.contains("q")]
#train = train[~train.libro.str.contains(" ")]
#train['libro'] = train['libro'].astype(int)
#print(len(train))
##Check para ver si son todos numericos
##train.applymap(np.isreal).sum()

In [11]:
train = df_train
test = df_test
train['libro'] = train['libro'].astype('category')
test['libro'] = test['libro'].astype('category')

In [12]:
full_x = train[['libro', 'usuario','svd','knn']]
full_y = train[['puntuacion']]
X_train, X_test, y_train, y_test = train_test_split(full_x, full_y, test_size=0.3, random_state=0)

In [13]:
test= test[['libro','usuario','svd','knn']]
test

,libro,usuario,svd,knn
0,los-hijos,201,7.17,7.17
1,el-temor-de-un-hombre-sabio,299,9.44,8.34
2,leon-bocanegra,126,7.41,7.48
3,el-caballero-errante-2,107,7.61,7.75
4,el-mar-el-mar,85,6.59,7.17
...,...,...,...,...
10579,la-princesa-prometida,854,8.57,8.67
10580,la-prima-bette,216,8.05,7.01
10581,el-guardian-invisible-trilogia-del-baztan-1,132,6.90,7.46
10582,un-otono-romano,133,9.08,9.11


In [14]:
X_test

,libro,usuario,svd,knn
36034,si-escucharas-mi-corazon,151,7.87,6.81
29252,la-mujer-del-camarote-10,162,7.92,7.54
11259,los-pilares-de-la-tierra-edicion-25-aniversario,128,8.04,7.71
16446,sputnik-mi-amor,43,8.61,7.18
10521,84-charing-cross-road,93,7.54,6.82
...,...,...,...,...
15153,la-celestina-comedia-o-tragicomedia-de-calisto...,312,5.75,5.65
5897,la-peregrina,61,2.39,6.51
6079,los-restos-del-dia,513,7.60,7.36
27797,las-tres-espanas-del-36,62,6.14,6.97


## LGB

In [15]:
# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [16]:
# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 50,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

In [17]:
print('Starting training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)
# save model to file
#gbm.save_model('model.txt')

Starting training...
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000192 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[1]	valid_0's l1: 1.27971	valid_0's l2: 2.73009
Training until validation scores don't improve for 5 rounds
[2]	valid_0's l1: 1.21682	valid_0's l2: 2.46891
[3]	valid_0's l1: 1.15715	valid_0's l2: 2.23322
[4]	valid_0's l1: 1.10084	valid_0's l2: 2.02051
[5]	valid_0's l1: 1.04742	valid_0's l2: 1.82844
[6]	valid_0's l1: 0.996701	valid_0's l2: 1.65508
[7]	valid_0's l1: 0.948481	valid_0's l2: 1.49857
[8]	valid_0's l1: 0.902778	valid_0's l2: 1.35726
[9]	valid_0's l1: 0.859328	valid_0's l2: 1.22967
[10]	valid_0's l1: 0.81803	valid_0's l2: 1.11448
[11]	valid_0's l1: 0.778914	valid_0's l2: 1.01075
[12]	valid_0's l1: 0.74182	valid_0's l2: 0.917104
[13]	valid_0's l1: 0.706637	valid_0's l2: 0.832581
[14]	valid_0's l1: 0.673361	valid_0's l2: 0.7563
[15]	

In [18]:
print('Starting predicting...')
# predict
y_pred_lgb = gbm.predict(X_test, num_iteration=gbm.best_iteration)
# eval
print('RMSE: %.4f' % np.sqrt(mean_squared_error(y_test, y_pred_lgb)))
#RMSE: 1.6434
# Ensamble:RMSE: 0.6602
# Ensamble:RMSE: 0.6557

Starting predicting...
RMSE: 0.6557


In [21]:
y_pred_lgb_test = gbm.predict(test, num_iteration=gbm.best_iteration)

In [22]:
y_pred_lgb_test

array([7.05691171, 8.97206508, 7.31606201, ..., 7.05139638, 8.43053397,
       5.61982039])

In [24]:
write_submit(df_test,y_pred_lgb_test, "ignacio_submit_ensamble2")

## XGBOOST

In [10]:
## No funciona con data Categorica
#model_xgb = XGBRegressor()
#model_xgb.fit(X_train, y_train)
#print('Starting predicting...')
#y_pred_xgb = model_xgb.predict(X_test)
#print('RMSE: %.4f' % np.sqrt(mean_squared_error(y_test, y_pred_xgb)))